# Example for ERA5 weather data download

This example shows you how to download ERA5 weather data from the [Climate Data Store (CDS)](https://cds.climate.copernicus.eu) and store it locally. Furthermore, it shows how to convert the weather data to the format needed by the pvlib and windpowerlib.

In order to download ERA5 weather data you need an account at the [CDS](https://cds.climate.copernicus.eu).
Furthermore, you need to install the cdsapi package. See [here](https://cds.climate.copernicus.eu/api-how-to) for installation details. 

When downloading the data using the API your request gets queued and may take a while to be completed. All actual calls of the data download are therefore commented to avoid unintended download. 

* [Download data for single coordinate](#single_loc)
* [Download data for a region](#region)
* [Convert data into pvlib and windpowerlib format](#convert)


In [1]:
from feedinlib import era5

## Download data for single coordinate <a class="anchor" id="single_loc"></a>

To download data for a single location you have to specify latitude and longitude of the desired location. Data will be retrieved for the nearest weather data point to that location.

In [2]:
latitude = 52.4
longitude = 13.5

Besides a location you have to specify a time period for which you would like to download the data as well as the weather variables you need. The feedinlib provides predefined sets of variables that are needed to use the pvlib and windpowerlib. These can be applied by setting the `variable` parameter to "pvlib" or "windpowerlib", as shown below. If you want to download data for both library you can set `variable` to 'feedinlib'.

In [3]:
# set start and end date (end date will be included 
# in the time period for which data is downloaded)
start_date, end_date = '2015-01-01', '2015-02-01'
# set variable set to download
variable = "windpowerlib"

If you want to store the downloaded data you may provide a filename (including path) to save data to.

In [4]:
target_file = 'ERA5_weather_data.nc'

Now we can retrieve the data:

```python
# get windpowerlib data for specified location
ds = era5.get_era5_data_from_datespan_and_position(
    variable=variable,
    start_date=start_date, end_date=end_date, 
    latitude=latitude, longitude=longitude,
    target_file=target_file)
```

## Download data for a region<a class="anchor" id="region"></a>

When wanting to download weather data for a region instead of providing a single value for each latitude and longitude you have to provide latitude and longitude as lists in the following form:

In [5]:
latitude = [51.0, 53.0]  # [latitude south, latitude north]
longitude = [13.5, 13.8]  # [longitude west, longitude east]

In [6]:
variable = "pvlib"

```python
# get pvlib data for specified area
ds = era5.get_era5_data_from_datespan_and_position(
    variable=variable,
    start_date=start_date, end_date=end_date, 
    latitude=latitude, longitude=longitude,
    target_file=target_file)
```

If you want weather data for the whole world, you may leave latitude and longitude unspecified. 

```python
# get feedinlib data (includes pvlib and windpowerlib data)
# for the whole world
ds = era5.get_era5_data_from_datespan_and_position(
    variable="feedinlib",
    start_date=start_date, end_date=end_date,
    target_file=target_file)
```

## Convert data into pvlib and windpowerlib format<a class="anchor" id="convert"></a>

In order to use the weather data for your feed-in calculations using the pvlib and windpowerlib it has to be converted into the required format. This can easily be done as follows. 

In [7]:
# filename (including path) downloaded ERA5 data was saved to
era5_netcdf_filename = 'ERA5_weather_data.nc'

```python
# get weather data in windpowerlib format for all locations in
# netcdf file
windpowerlib_df = era5.weather_df_from_era5(
    era5_netcdf_filename=era5_netcdf_filename,
    lib='windpowerlib')
```

If the netcdf contains more than one location the `windpowerlib_df` will contain all of those locations and the index of the dataframe will be a multiindex with levels (time, latitude, longitude). To use it for windpowerlib calculations you need to select a **single location** first.
To directly obtain data for a single location you need to specify it using the `area` parameter:

```python
# get weather data in windpowerlib format for single location
# (weather data for nearest weather data point to specified
# location is returned)
area = [13.5, 52.4]
windpowerlib_df = era5.weather_df_from_era5(
    era5_netcdf_filename=era5_netcdf_filename,
    lib='windpowerlib', area=area)
```

You may also specify an area for which to retrieve weather data. Again, keep in mind that in that case the index of the returned dataframe will be a multiindex with levels (time, latitude, longitude) and cannot be directly used for windpowerlib or pvlib calculations.

In [8]:
# specify rectangular area
area = [(13.5, 13.8), (51.0, 53.0)]

In [9]:
# specify area giving a Polygon
from shapely.geometry import Polygon
lat_point_list = [51.936377, 51.936377, 51.744302, 51.744302, 51.936377]
lon_point_list = [12.621739, 13.005414, 13.005414, 12.621739, 12.621739]
area = Polygon(zip(lon_point_list, lat_point_list))

Furthermore, it is possible to specify a start and end date to retrieve data for. They must be provided as something that can be converted to a timestamp, i.e. '2013-07-02'.

```python
# get weather data in pvlib format starting January 15th
start = '2015-01-15'
windpowerlib_df = era5.weather_df_from_era5(
    era5_netcdf_filename=era5_netcdf_filename,
    lib='pvlib', area=area, start=start)
```